# Creating a network with a SmartNIC

This Jupyter notebook will walk you through creating an experiment that contains a SmartNIC. The SmartNIC is Connectx-6, from NVIDIA (Mellanox). Specifically, the experiment deploys a slice with two nodes: one server with a SmartNIC connected to another server with a basic NIC.


# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](./prepare_env.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice

In [2]:
slice = fablib.new_slice(name="p4sec_fabric_samesite")
#slice = fablib.get_slice(name="smartnic222")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: MICH, STAR, and NCSA

<img src="./figs/fabric_sites.png" width="40%"><br>

In [3]:
# site1='FIU'
# site2='FIU'

cx5_column_name = 'nic_connectx_6_available'

# find two sites with available ConnectX-5 
# sites = fablib.get_random_sites(count=1, filter_function=lambda x: x[cx5_column_name] > 2)

#site1 = sites[0]
#site2 = sites[0]

site = 'SEAT'
site1= site
site2= site
site3= site


print (f'The selected sites are {site1}, {site2}, {site3}') 

The selected sites are SEAT, SEAT, SEAT


### Step 3.3: Creating the nodes
The code below creates three nodes: server1, switch, and server2. The servers (server1 and server2) use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server3 will be created in site3

<img src="./figs/creating_nodes.PNG" width="40%"><br>

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=24, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site2, 
                      cores=24, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
server3 = slice.add_node(name="server3", 
                      site=site3, 
                      cores=24, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./figs/adding_nics.PNG" width="40%"><br>

In [5]:
server1_iface = server1.add_component(model='NIC_ConnectX_6', name='nic').get_interfaces()[0]
server21_iface = server2.add_component(model='NIC_ConnectX_6', name='nic2').get_interfaces()[0]
server23_iface = server2.get_component(name='nic2').get_interfaces()[1]
server3_iface = server3.add_component(model='NIC_ConnectX_6', name='nic3').get_interfaces()[0]

In [6]:
# print(server2.get_component(name='nic2').get_interfaces()[0])

In [7]:
# print(server2.get_component(name='nic2').get_interfaces()[1])

### Step 3.5: Connecting site1 and site2 and site3
Create a site-to-site network between site1 and site2 connecting server1 and the P4 switch

<img src="./figs/connecting_nodes_server1_switch.PNG" width="40%"><br>

In [8]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, server21_iface])
net2 = slice.add_l2network(name='net2', interfaces=[server23_iface, server3_iface])

### Step 3.6: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [9]:
#Submit Slice Request
slice.submit();


Retry: 17, Time: 4018 sec


ID,9cd180a2-19cb-41a7-b23d-ed864f606cdc
Name,p4sec_fabric_samesite
Lease Expiration (UTC),2024-04-02 15:47:56 +0000
Lease Start (UTC),2024-04-01 15:47:57 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e6792304-760d-43b2-9339-7ff84c9f4f2c,server1,24,8,100,default_ubuntu_20,qcow2,seat-w1.fabric-testbed.net,SEAT,ubuntu,2001:400:a100:3080:f816:3eff:fed9:d0f2,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fed9:d0f2,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ee617c2d-e768-4f5e-955a-6e4a0238a449,server2,24,8,100,default_ubuntu_20,qcow2,seat-w1.fabric-testbed.net,SEAT,ubuntu,2001:400:a100:3080:f816:3eff:fee0:d0d6,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fee0:d0d6,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
18f5e112-c805-4580-94eb-bf13f903e00d,server3,24,8,100,default_ubuntu_20,qcow2,seat-w1.fabric-testbed.net,SEAT,ubuntu,2001:400:a100:3080:f816:3eff:fe88:dd85,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe88:dd85,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
87d698ff-468c-4aec-a6dd-bdc538dce3b5,net1,L2,L2Bridge,SEAT,None,None,Active,
443e8ffd-e883-4451-9c2e-2ddedb9676a2,net2,L2,L2Bridge,SEAT,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
server1-nic-p1,p1,server1,net1,100,config,,10:70:FD:E5:EC:40,None,None,None,6
server1-nic-p2,p2,server1,None,100,config,,10:70:FD:E5:EC:41,None,None,None,6
server2-nic2-p2,p2,server2,net2,100,config,,10:70:FD:E5:E7:79,None,None,None,2
server2-nic2-p1,p1,server2,net1,100,config,,10:70:FD:E5:E7:78,None,None,None,2
server3-nic3-p1,p1,server3,net2,100,config,,10:70:FD:E5:E5:F8,None,None,None,1
server3-nic3-p2,p2,server3,None,100,config,,10:70:FD:E5:E5:F9,None,None,None,1



Time to print interfaces 4446 seconds


# Step 4: Set the workspace

These steps apply to all servers  

### Upload .sh file (ask)

In [3]:
from ipaddress import ip_address, IPv6Address

server1 = slice.get_node(name="server1")     

# If the node is an IPv6 Node then configure NAT64
if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh')

server1.upload_file('host_tune.sh', 'host_tune.sh')
stdout, stderr = server1.execute(f'chmod +x host_tune.sh && sudo ./host_tune.sh')

AuthenticationException: Authentication failed.

In [13]:
from ipaddress import ip_address, IPv6Address

server2 = slice.get_node(name="server2")     

# If the node is an IPv6 Node then configure NAT64
if type(ip_address(server2.get_management_ip())) is IPv6Address:
    server2.upload_file('nat64.sh', 'nat64.sh')

    stdout, stderr = server2.execute(f'chmod +x nat64.sh && ./nat64.sh')

server2.upload_file('host_tune.sh', 'host_tune.sh')
stdout, stderr = server2.execute(f'chmod +x host_tune.sh && sudo ./host_tune.sh')

 sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Temporary failure in name resolution
sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
 * Applying /etc/sysctl.d/10-console-messages.conf ...
kernel.printk = 4 4 1 7
* Applying /etc/sysctl.d/10-ipv6-privacy.conf ...
net.ipv6.conf.all.use_tempaddr = 2
net.ipv6.conf.default.use_tempaddr = 2
* Applying /etc/sysctl.d/10-kernel-hardening.conf ...
kernel.kptr_restrict = 1
* Applying /etc/sysctl.d/10-link-restrictions.conf ...
fs.protected_hardlinks = 1
fs.protected_symlinks = 1
* Applying /etc/sysctl.d/10-magic-sysrq.conf ...
kernel.sysrq = 176
* Applying /etc/sysctl.d/10-network-security.conf ...
net.ipv4.conf.default.rp_filter = 2
net.ipv4.conf.all.rp_filter = 2
* Applying /etc/sysctl.d/10-ptrace.conf ...
kernel.yama.ptrace_scope = 1
* Applying /etc/sysctl.d/10-zeropage.conf ...
vm.mmap_min_addr = 65536
* Applyi

In [14]:
from ipaddress import ip_address, IPv6Address

server3 = slice.get_node(name="server3")     

# If the node is an IPv6 Node then configure NAT64
if type(ip_address(server3.get_management_ip())) is IPv6Address:
    server3.upload_file('nat64.sh', 'nat64.sh')

    stdout, stderr = server3.execute(f'chmod +x nat64.sh && ./nat64.sh')

server3.upload_file('host_tune.sh', 'host_tune.sh')
stdout, stderr = server3.execute(f'chmod +x host_tune.sh && sudo ./host_tune.sh')

 sudo: unable to resolve host server3: Name or service not known
sudo: unable to resolve host server3: Temporary failure in name resolution
sudo: unable to resolve host server3: Name or service not known
sudo: unable to resolve host server3: Name or service not known
 * Applying /etc/sysctl.d/10-console-messages.conf ...
kernel.printk = 4 4 1 7
* Applying /etc/sysctl.d/10-ipv6-privacy.conf ...
net.ipv6.conf.all.use_tempaddr = 2
net.ipv6.conf.default.use_tempaddr = 2
* Applying /etc/sysctl.d/10-kernel-hardening.conf ...
kernel.kptr_restrict = 1
* Applying /etc/sysctl.d/10-link-restrictions.conf ...
fs.protected_hardlinks = 1
fs.protected_symlinks = 1
* Applying /etc/sysctl.d/10-magic-sysrq.conf ...
kernel.sysrq = 176
* Applying /etc/sysctl.d/10-network-security.conf ...
net.ipv4.conf.default.rp_filter = 2
net.ipv4.conf.all.rp_filter = 2
* Applying /etc/sysctl.d/10-ptrace.conf ...
kernel.yama.ptrace_scope = 1
* Applying /etc/sysctl.d/10-zeropage.conf ...
vm.mmap_min_addr = 65536
* Applyi

### Pinning numa nodes

In [6]:
server1.pin_cpu(component_name='nic')
server2.pin_cpu(component_name='nic2')
server3.pin_cpu(component_name='nic3')
    
# User can also pass in the range of the vCPUs to be pinned 
#node.pin_cpu(component_name=nic_name, cpu_range_to_pin="0-3")
    
# Pin memmory for VM to same Numa node as the components
server1.numa_tune()
server2.numa_tune()
server3.numa_tune()
        
# Reboot the VM
# .os_reboot()

Pinning Node: server1 CPUs for component: nic to Numa Node: 2
Pinning Node: server2 CPUs for component: nic2 to Numa Node: 1
Pinning Node: server3 CPUs for component: nic3 to Numa Node: 6
Numa tune Node: server1 Memory to Numa  Nodes: ['2']


Exception: POA - f909d61b-e29a-4ebe-a5e0-239b8b6c1da5/numatune failed with error: - Exception during poa for unit: c384c49b-694f-4c4c-8ff2-a65c91959b09 msg Playbook has failed tasks: non-zero return code

### Reboot servers

In [ ]:
server1.os_reboot()
server2.os_reboot()
server3.os_reboot()

### Install needed dependencies

In [15]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y python3-pip', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install -y python3-pyelftools', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install -y libnuma-dev', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install -y pkg-config', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install net-tools', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install iperf3', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install hping3', quiet = True)
stdout, stderr = server1.execute(f'sudo apt install tcpdump', quiet = True)

#stdout, stderr = server1.execute(f'sudo echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages')

In [16]:
stdout, stderr = server2.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server2.execute(f'sudo apt-get install -y build-essential', quiet = True)
stdout, stderr = server2.execute(f'sudo apt-get install -y python3-pip', quiet = True)
stdout, stderr = server2.execute(f'sudo pip3 install meson ninja', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install -y python3-pyelftools', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install -y libnuma-dev', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install -y pkg-config', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install net-tools', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install iperf3', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install hping3', quiet = True)
stdout, stderr = server2.execute(f'sudo apt install tcpdump', quiet = True)

#stdout, stderr = server1.execute(f'sudo echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages')

In [17]:
stdout, stderr = server3.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server3.execute(f'sudo apt-get install -y build-essential', quiet = True)
stdout, stderr = server3.execute(f'sudo apt-get install -y python3-pip', quiet = True)
stdout, stderr = server3.execute(f'sudo pip3 install meson ninja', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install -y python3-pyelftools', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install -y libnuma-dev', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install -y pkg-config', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install net-tools', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install iperf3', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install hping3', quiet = True)
stdout, stderr = server3.execute(f'sudo apt install tcpdump', quiet = True)

#stdout, stderr = server1.execute(f'sudo echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages')

### Download, unzip and install Mellanox libraries and enable DPDK

In [18]:
stdout, stderr = server1.execute(f'wget https://content.mellanox.com/ofed/MLNX_OFED-23.07-0.5.0.0/MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz', quiet = True)
stdout, stderr = server1.execute(f'tar xvfz MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz', quiet=True)
stdout, stderr = server1.execute(f'cd MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64 && echo "y" | sudo ./mlnxofedinstall --upstream-libs --dpdk --basic --without-fw-update', quiet=True)

In [3]:
stdout, stderr = server2.execute(f'wget https://content.mellanox.com/ofed/MLNX_OFED-23.07-0.5.0.0/MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz', quiet = True)
stdout, stderr = server2.execute(f'tar xvfz MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz', quiet=True)
stdout, stderr = server2.execute(f'cd MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64 && echo "y" | sudo ./mlnxofedinstall --upstream-libs --dpdk --basic --without-fw-update', quiet=True)

In [20]:
stdout, stderr = server3.execute(f'wget https://content.mellanox.com/ofed/MLNX_OFED-23.07-0.5.0.0/MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz', quiet = True)
stdout, stderr = server3.execute(f'tar xvfz MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz', quiet=True)
stdout, stderr = server3.execute(f'cd MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64 && echo "y" | sudo ./mlnxofedinstall --upstream-libs --dpdk --basic --without-fw-update', quiet=True)

### Clone, build and install DPDK

In [21]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

In [4]:
stdout, stderr = server2.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server2.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

In [23]:
stdout, stderr = server3.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server3.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

### Restrat to enable configuration (mlnx)

In [24]:
stdout, stderr = server1.execute(f'sudo /etc/init.d/openibd restart')

Unloading HCA driver:[  OK  ]
 sudo: unable to resolve host server1: Name or service not known
 Loading HCA driver and Access Layer:[  OK  ]


In [5]:
stdout, stderr = server2.execute(f'sudo /etc/init.d/openibd restart')

Unloading HCA driver:[  OK  ]
 sudo: unable to resolve host server2: Name or service not known
 Loading HCA driver and Access Layer:[  OK  ]


In [26]:
stdout, stderr = server3.execute(f'sudo /etc/init.d/openibd restart')

Unloading HCA driver: sudo: unable to resolve host server3: Name or service not known
 [  OK  ]
Loading HCA driver and Access Layer:[  OK  ]


# Step 5: Setup pipeline on server 2

## Run the Cmake file of the dpdk pipeline

In [6]:
stdout, stderr = server2.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

## Upload the needed files to run the pipeline to the server
Before running this block make sure you edit the .io and .cli files

In [28]:
# server2.upload_file('test.p4','test.p4')
# server2.upload_file('test.cli','test.cli')
# server2.upload_file('ethdev.io','ethdev.io')

## Setup the p4c compiler

In [7]:
stdout, stderr = server2.execute('source /etc/lsb-release && echo "deb http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${DISTRIB_RELEASE}/ /" | sudo tee /etc/apt/sources.list.d/home:p4lang.list && curl -fsSL https://download.opensuse.org/repositories/home:p4lang/xUbuntu_${DISTRIB_RELEASE}/Release.key | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/home_p4lang.gpg > /dev/null && sudo apt-get update && sudo apt install -y p4lang-p4c', quiet = True)

In [30]:
# stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=psa test.p4 -o test.spec && cp test.spec dpdk/examples/pipeline/examples/ && cp ethdev.io dpdk/examples/pipeline/examples/')
# stdout, stderr = server1.execute(f'cp test.cli dpdk/examples/pipeline/examples/')

## Reserve memory resources

In [46]:
stdout, stderr = server1.execute(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"')

 sudo: unable to resolve host server1: Name or service not known
 

2048

In [5]:
stdout, stderr = server2.execute(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"')

 sudo: unable to resolve host server2: Name or service not known
 

In [33]:
stdout, stderr = server3.execute(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"')

 sudo: unable to resolve host server3: Name or service not known
 

## (in server 2 terminal) run MLNX Commands

In [40]:
# stdout, stderr = server2.execute(f'sudo ibdev2netdev')
# stdout, stderr = server2.execute(f'sudo mst status')
# stdout, stderr = server2.execute(f'sudo mst start')
# stdout, stderr = server2.execute(f'sudo mst status')

# Step 6: Setup communication requirements on servers

## Set IP Addresses on all servers
use "ifconfig" to see turned-up interfaces

use "lspci" to see connected interfaces 

use "ip link" to know the correct interface name

use "ip route" to see routing table

use "arp" to see arp routes

In [12]:
server1 = slice.get_node(name="server1")     

stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server1.execute(f'sudo ip link set dev enp7s0np0 up')
stdout, stderr = server1.execute(f'sudo ifconfig enp7s0np0 198.18.0.10/24')
stdout, stderr = server1.execute(f'sudo ifconfig enp7s0np0 hw ether 00:10:00:00:00:00')
stdout, stderr = server1.execute(f'sudo arp -s 198.18.0.20 00:21:00:00:00:00')
#stdout, stderr = server1.execute(f'sudo hping3 198.18.0.20 --flood --data 1460') #use multi process, nload, htop


 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
 

In [9]:
server2 = slice.get_node(name="server2")     

stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y net-tools', quiet=True)

stdout, stderr = server2.execute(f'sudo ip link set dev enp7s0np0 up')
stdout, stderr = server2.execute(f'sudo ifconfig enp7s0np0 198.18.0.20/24')
stdout, stderr = server2.execute(f'sudo ifconfig enp7s0np0 hw ether 00:21:00:00:00:00')

stdout, stderr = server2.execute(f'sudo ip link set dev enp8s0np1 up')
stdout, stderr = server2.execute(f'sudo ifconfig enp8s0np1 198.18.1.20/24')
stdout, stderr = server2.execute(f'sudo ifconfig enp8s0np1 hw ether 00:22:00:00:00:00')

stdout, stderr = server2.execute(f'sudo arp -s 198.18.0.10 00:10:00:00:00:00')
stdout, stderr = server2.execute(f'sudo arp -s 198.18.1.30 00:30:00:00:00:00')
#stdout, stderr = server2.execute(f'sudo hping3 198.18.0.20 --flood --data 1460') #use multi process, nload, htop


 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [13]:
server3 = slice.get_node(name="server3")     

stdout, stderr = server3.execute(f'sudo apt-get update && sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server3.execute(f'sudo ip link set dev enp7s0np0 up')
stdout, stderr = server3.execute(f'sudo ifconfig enp7s0np0 198.18.1.30/24')
stdout, stderr = server3.execute(f'sudo ifconfig enp7s0np0 hw ether 00:30:00:00:00:00')
stdout, stderr = server3.execute(f'sudo arp -s 198.18.1.20 00:22:00:00:00:00')
#stdout, stderr = server3.execute(f'sudo hping3 198.18.0.20 --flood --data 1460') #use multi process, nload, htop


 sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
 

In [45]:
# stdout, stderr = server3.execute(f'sudo ip link set dev enp8s0np1 down')
# stdout, stderr = server3.execute(f'sudo ifconfig enp8s0np1 198.18.2.30/24')
# stdout, stderr = server3.execute(f'sudo ifconfig enp8s0np1 hw ether 00:32:00:00:00:00')
# stdout, stderr = server3.execute(f'sudo arp -s 198.18.1.20 00:22:00:00:00:00')

## Clone Pktgen-DPDK on servers 1 & 3

In [38]:
stdout, stderr = server1.execute(f'sudo git clone https://github.com/pktgen/Pktgen-DPDK')
stdout, stderr = server1.execute("sudo sed -i '40s/.*/deps += [dependency('\'numa\'', required: false)]/' /home/ubuntu/Pktgen-DPDK/app/meson.build")

 sudo: unable to resolve host server1: Name or service not known
Cloning into 'Pktgen-DPDK'...
  sudo: unable to resolve host server1: Name or service not known
 

In [39]:
stdout, stderr = server3.execute(f'sudo git clone https://github.com/pktgen/Pktgen-DPDK')
stdout, stderr = server3.execute("sudo sed -i '40s/.*/deps += [dependency('\'numa\'', required: false)]/' /home/ubuntu/Pktgen-DPDK/app/meson.build")

 sudo: unable to resolve host server3: Name or service not known
Cloning into 'Pktgen-DPDK'...
  sudo: unable to resolve host server3: Name or service not known
 

For the second instruction, make sure there is single quotes around numa in the file (to make changes, run in server 1terminal: cd Pktgen-DPDK/app && sudo vim meson.build)

In [40]:
stdout, stderr = server1.execute(f'sudo apt-get install -y cmake')
stdout, stderr = server1.execute(f'sudo apt-get install -y libpcap-dev')
stdout, stderr = server1.execute(f'cd Pktgen-DPDK &&  sudo meson build && sudo ninja -C build && cd build/ && sudo meson install')

Reading package lists... sudo: unable to resolve host server1: Name or service not known
 
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cmake-data libjsoncpp1 librhash0
Suggested packages:
  cmake-doc ninja-build
The following NEW packages will be installed:
  cmake cmake-data libjsoncpp1 librhash0
0 upgraded, 4 newly installed, 0 to remove and 53 not upgraded.
Need to get 5470 kB of archives.
After this operation, 28.3 MB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 cmake-data all 3.16.3-1ubuntu1.20.04.1 [1613 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 libjsoncpp1 amd64 1.7.4-3.1ubuntu2 [75.6 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 librhash0 amd64 1.3.9-1 [113 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 cmake amd64 3.16.3-1ubuntu1.20.04.1 [3668 kB]
 debco

In [48]:
stdout, stderr = server3.execute(f'sudo apt-get install -y cmake')
stdout, stderr = server3.execute(f'sudo apt-get install -y libpcap-dev')
stdout, stderr = server3.execute(f'cd Pktgen-DPDK &&  sudo meson build && sudo ninja -C build && cd build/ && sudo meson install')

Reading package lists...
 sudo: unable to resolve host server3: Name or service not known
 Building dependency tree...
Reading state information...
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.
Reading package lists...
Building dependency tree... sudo: unable to resolve host server3: Name or service not known
 
Reading state information...
libpcap-dev is already the newest version (1.9.1-3).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.
Directory already configured.

Just run your build command (e.g. ninja) and Meson will regenerate as necessary.
Run "meson setup --reconfigure to force Meson to regenerate.

If build failures persist, run "meson setup --wipe" to rebuild from scratch
using the same options as passed when configuring the build.
 sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
 ninja: Entering d

# Step 7: Upload and compile

In [36]:
server1.upload_file('ethtool.sh','ethtool.sh')
server3.upload_file('ethtool.sh','ethtool.sh')

<SFTPAttributes: [ size=309 uid=1000 gid=1000 mode=0o100664 atime=1710113975 mtime=1710214184 ]>

## To upload and compile PSA P4 example: test_PSA_1.p4

In [14]:
server2.upload_file('test_PSA_1p0.p4','test.p4')
server2.upload_file('test.cli','test.cli')
server2.upload_file('ethdev.io','ethdev.io')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=psa test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/ && sudo cp ethdev.io dpdk/examples/pipeline/examples/ ')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [16]:
server2.upload_file('test_PSA_1p1.p4','test.p4')
server2.upload_file('test.cli','test.cli')
server2.upload_file('ethdev.io','ethdev.io')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=psa test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/ && sudo cp ethdev.io dpdk/examples/pipeline/examples/ ')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [17]:
server2.upload_file('test_PSA_2.p4','test.p4')
server2.upload_file('test.cli','test.cli')
server2.upload_file('ethdev.io','ethdev.io')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=psa test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/ && sudo cp ethdev.io dpdk/examples/pipeline/examples/ ')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

## To upload and compile PNA P4 example: test_PNA.p4

In [11]:
server2.upload_file('test_PNA.p4','test.p4')
server2.upload_file('test.cli','test.cli')
server2.upload_file('ethdev.io','ethdev.io')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/ && sudo cp ethdev.io dpdk/examples/pipeline/examples/ ')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

## To upload and compile cms: cms.p4

In [47]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('test.cli','test.cli')
server2.upload_file('ethdev.io','ethdev.io')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/ && sudo cp ethdev.io dpdk/examples/pipeline/examples/ ')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

## To upload and compile cms different hashes: cms_hx.p4

In [24]:
server2.upload_file('cms_h8.p4','test.p4')
server2.upload_file('multicore/2cores/test.cli','test.cli')

server2.upload_file('multicore/2cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/2cores/ethdev2.io','ethdev2.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [37]:
server2.upload_file('cms_h5.p4','test.p4')
server2.upload_file('multicore/4cores/test.cli','test.cli')

server2.upload_file('multicore/4cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/4cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/4cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/4cores/ethdev4.io','ethdev4.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

## To upload and compile 4cores cms: cms.p4

In [2]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/4cores/test.cli','test.cli')

server2.upload_file('multicore/4cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/4cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/4cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/4cores/ethdev4.io','ethdev4.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

<br> 
<br>
<br>
<br>
<br>
<br> 
<br>
<br>
<br>
<br>
<br> 
<br>
<br>
<br>
<br>
<br> 
<br>
<br>
<br>
<br>
<br> 
<br>
<br>
<br>
<br>
<br> 
<br>
<br>
<br>
<br>
  

## To upload and compile multicore test: cms_optimal.p4

1 core 1 pipeline

In [6]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/1core/test.cli','test.cli')

server2.upload_file('multicore/1core/ethdev.io','ethdev.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

2 cores 2 pipelines

In [4]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/2cores/test.cli','test.cli')

server2.upload_file('multicore/2cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/2cores/ethdev2.io','ethdev2.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

3 cores 3 pipelines

In [8]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/3cores/test.cli','test.cli')

server2.upload_file('multicore/3cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/3cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/3cores/ethdev3.io','ethdev3.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

4 cores 4 pipelines

In [3]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/4cores/test.cli','test.cli')

server2.upload_file('multicore/4cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/4cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/4cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/4cores/ethdev4.io','ethdev4.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

5 cores 5 pipelines

In [15]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/5cores/test.cli','test.cli')

server2.upload_file('multicore/5cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/5cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/5cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/5cores/ethdev4.io','ethdev4.io')
server2.upload_file('multicore/5cores/ethdev5.io','ethdev5.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev5.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

6 cores 6 pipelines

In [16]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/6cores/test.cli','test.cli')

server2.upload_file('multicore/6cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/6cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/6cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/6cores/ethdev4.io','ethdev4.io')
server2.upload_file('multicore/6cores/ethdev5.io','ethdev5.io')
server2.upload_file('multicore/6cores/ethdev6.io','ethdev6.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev5.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev6.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

7 cores 7 pipelines

In [17]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/7cores/test.cli','test.cli')

server2.upload_file('multicore/7cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/7cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/7cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/7cores/ethdev4.io','ethdev4.io')
server2.upload_file('multicore/7cores/ethdev5.io','ethdev5.io')
server2.upload_file('multicore/7cores/ethdev6.io','ethdev6.io')
server2.upload_file('multicore/7cores/ethdev7.io','ethdev7.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev5.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev6.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev7.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

8 cores 8 pipelines

In [18]:
server2.upload_file('cms_optimal.p4','test.p4')
server2.upload_file('multicore/8cores/test.cli','test.cli')

server2.upload_file('multicore/8cores/ethdev.io','ethdev.io')
server2.upload_file('multicore/8cores/ethdev2.io','ethdev2.io')
server2.upload_file('multicore/8cores/ethdev3.io','ethdev3.io')
server2.upload_file('multicore/8cores/ethdev4.io','ethdev4.io')
server2.upload_file('multicore/8cores/ethdev5.io','ethdev5.io')
server2.upload_file('multicore/8cores/ethdev6.io','ethdev6.io')
server2.upload_file('multicore/8cores/ethdev7.io','ethdev7.io')
server2.upload_file('multicore/8cores/ethdev8.io','ethdev8.io')

stdout, stderr = server2.execute(f'sudo cp ethdev.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev2.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev3.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev4.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev5.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev6.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev7.io dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp ethdev8.io dpdk/examples/pipeline/examples/')

stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/')
stdout, stderr = server2.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

# Step 7: Running the DPDK application on the SmartNIC

## These commands should be exectided in the terminals

Make sure to export RTE_INSTALL_DIR=/home/ubuntu/dpdk (ask)

In [50]:
# stdout, stderr = server1.execute(f'cd dpdk && sudo examples/pipeline/build/pipeline -c 0xF -- -s examples/pipeline/examples/test.cli')

### Run Pktgen in server 1

In [ ]:
stdout, stderr = server1.execute(f'sudo sudo pktgen -l 0-3 -n 3 -- -P -m "1.0"')

In [ ]:
# stdout, stderr = server1.execute(f'sudo pktgen -l 0-8 -n 7 -- -p -m "[1:7].0"')

In [ ]:
# sudo sysctl -w vm.nr_hugepages=102400
# sudo su
# echo 32768 > /proc/sys/vm/nr_hugepages
# exit
# sudo pktgen -c 0x1F -- -P -m "[1-4].0"
stdout, stderr = server1.execute(f'sudo pktgen -c 0xF -- -P -m "[1-4].0"')

In Pktgen set mac ip and packet size

source mac: set 0 src mac {mac address}

destination mac: set 0 dst mac {mac address}

source ip: set 0 src ip {ip address}

destination ip: set 0 dst ip {ip address}

packet size: set 0 size {packet size}

%Rate out of 100 Gbps: set 0 rate {rate/100}

strat 0
stop 0

### Run pipeline in server 2

In [ ]:
stdout, stderr = server2.execute(f'cd dpdk')

In [ ]:
stdout, stderr = server2.execute(f'sudo examples/pipeline/build/pipeline -c 0xF -- -s examples/pipeline/examples/test.cli')

In [ ]:
stdout, stderr = server2.execute(f'sudo examples/pipeline/build/pipeline -c 0x1F -- -s examples/pipeline/examples/test.cli')

t2 >> telnet 0.0.0.0 8086

pipeline >> ethdev show
ethtool -S eth0 | grep -E 'tx_packets|rx_packets'

### Run in server 3 (option 1)

t1 >> sudo sysctl -w net.ipv4.tcp_congestion_control=cubic >> sudo hping3 198.18.1.20 -i u10000

t2 >> iperf3 -s

t3 >> iperf3 -c 198.18.1.20 -t 60

----
use tcpdump to see all incomming and outgoing transactions : sudo tcpdump -i enp7s0np0

# When the session is extended and the work needs to be resumed

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

slice = fablib.get_slice(name="p4sec_fabric_samesite")

server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
server3 = slice.get_node(name="server3")

# server1.upload_file('test.p4','test.p4')
# server1.upload_file('test.cli','test.cli')
# server1.upload_file('ethdev.io','ethdev.io')
# stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=psa test.p4 -o test.spec && sudo cp test.spec dpdk/examples/pipeline/examples/ && sudo cp ethdev.io dpdk/examples/pipeline/examples/')
# stdout, stderr = server1.execute(f'sudo cp test.cli dpdk/examples/pipeline/examples/')

